## **Sales Forecasting of Retail Clothing Product Categories**

This code creates a csv file, with input parameters and output predictions . The csv file can be used for dashboard visualization

Importing required libraries

In [20]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import joblib

Load Config

In [21]:
import yaml

#read yaml file
with open('../config/config.yaml') as file:
  config= yaml.safe_load(file)
  #print(config)

## Load Processed dataset

Get the Root Dir

In [22]:
#Get current working dir
cwd_path=os.getcwd()
#print(cwd_path)

#  Get the path of ROOT DIR
path_list = cwd_path.split(os.sep) #split path
ROOT_PATH=os.path.join(path_list[0],os.sep,*path_list[1:-2]) # To move two level up, remove last two str and join to create root path
#print(ROOT_PATH)

Read the clean & Merged data

In [23]:
# process Data file
Process_Data_file=os.path.join(ROOT_PATH,config['processed_data_dir'],config['processed_data_file_1'])
print(Process_Data_file)

# Read the Sales input file name
try:
    Data=pd.read_csv(Process_Data_file)
    #sales_data.head()
except FileNotFoundError:
    print("file {} does not exist".format(Process_Data_file))



c:\Nishikant\OneDrive - CriticalRiver Technologies Pvt. Ltd\Development\1-Team Projects\AIML_Projects\Retail Sales Forecasting\data\processed\Merged_data.csv


In [24]:
Data=Data.sample(n =100)

In [25]:
Data.reset_index(drop=True, inplace=True)

In [26]:
Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,MenClothing,645.0,17900.981857,16329.523792,259.647,5.4,11.98,9.85,25.04,69.35,...,10.250,3.516129,-6.096774,54.000000,1019.032258,12.935484,9.838710,3.868065,0,0
1,WomenClothing,4775.0,15740.544526,15148.822195,248.307,8.5,12.36,10.36,23.22,95.45,...,10.650,6.500000,-0.566667,62.500000,1020.633333,14.666667,9.533333,3.386333,2,2
2,WomenClothing,3646.0,17931.046190,16400.926737,259.240,5.5,11.98,9.85,25.05,69.84,...,10.200,-4.250000,-12.428571,53.892857,1017.607143,13.428571,11.750000,1.851786,1,1
3,MenClothing,599.0,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,...,15.280,5.838710,-3.387097,56.096774,1019.612903,14.000000,12.290323,5.072258,0,0
4,OtherClothing,1566.0,17061.071077,15810.410182,257.284,6.7,11.85,10.22,24.30,87.49,...,9.650,3.709677,-3.000000,61.903226,1019.225806,12.935484,9.419355,3.974839,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WomenClothing,2164.0,14317.372922,14345.676097,237.600,9.5,13.32,11.25,22.00,64.80,...,9.700,22.741935,16.451613,70.806452,1013.580645,12.935484,7.903226,5.825806,0,2
96,OtherClothing,1468.0,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,...,9.450,7.700000,-1.800000,54.066667,1017.400000,14.300000,8.600000,3.598667,0,2
97,MenClothing,599.0,16503.915684,15554.426878,254.807,8.0,11.88,10.64,23.88,85.51,...,11.475,1.935484,-5.419355,61.032258,1019.806452,13.258065,10.322581,2.262581,0,2
98,WomenClothing,3414.0,15488.004864,15050.661172,246.489,9.1,13.44,10.98,22.96,216.62,...,15.280,12.500000,4.966667,64.466667,1013.566667,13.066667,11.766667,4.530333,1,0


In [27]:
Data= Data.drop(['Averageuplandharvestedmillionacres','Productionin480lbnetwerightinmillionbales',
                                               'Tempavg','DewPointavg','Humidityavg','SalesInThousandDollars'],axis=1)

In [28]:
SampleData=Data.copy()

#### Processing user input

In [29]:
## converting text data into numerical
Data.replace({'MenClothing': 0, 'WomenClothing': 1,'OtherClothing':2},inplace=True)

In [30]:
Data

,ProductCategory,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,Changein,Averageuplandplantedmillionacres,yieldperharvestedacre,Millusein480lbnetwerightinmillionbales,Exports,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,0,17900.981857,16329.523792,259.647,5.4,11.98,9.85,25.04,69.35,-0.70,10.845,781,3.625,10.250,1019.032258,12.935484,9.838710,3.868065,0,0
1,1,15740.544526,15148.822195,248.307,8.5,12.36,10.36,23.22,95.45,-8.82,14.431,757,3.575,10.650,1020.633333,14.666667,9.533333,3.386333,2,2
2,1,17931.046190,16400.926737,259.240,5.5,11.98,9.85,25.05,69.84,3.70,10.845,781,3.625,10.200,1017.607143,13.428571,11.750000,1.851786,1,1
3,0,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,7.74,10.769,814,3.565,15.280,1019.612903,14.000000,12.290323,5.072258,0,0
4,2,17061.071077,15810.410182,257.284,6.7,11.85,10.22,24.30,87.49,3.35,10.136,788,3.580,9.650,1019.225806,12.935484,9.419355,3.974839,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,14317.372922,14345.676097,237.600,9.5,13.32,11.25,22.00,64.80,5.55,8.905,796,3.470,9.700,1013.580645,12.935484,7.903226,5.825806,0,2
96,2,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,-3.99,10.818,783,3.775,9.450,1017.400000,14.300000,8.600000,3.598667,0,2
97,0,16503.915684,15554.426878,254.807,8.0,11.88,10.64,23.88,85.51,2.57,12.077,849,3.380,11.475,1019.806452,13.258065,10.322581,2.262581,0,2
98,1,15488.004864,15050.661172,246.489,9.1,13.44,10.98,22.96,216.62,-5.68,10.769,804,3.665,15.280,1013.566667,13.066667,11.766667,4.530333,1,0


## Load the model

In [31]:
# Prediction model filename
prediction_model_file=os.path.join(ROOT_PATH,config['models_dir'],config['prediction_model'])
#print(prediction_model_file)

In [32]:
## load the model
import pickle
prediction_model = pickle.load(open(prediction_model_file, 'rb'))

In [33]:
## lets predict the labels for x data

y_pred=prediction_model.predict(Data)
y_pred

c:\Users\Nishikant\anaconda3\envs\cr_Retail_Sales\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


array([ 784.5278 , 4683.3115 , 3551.45   ,  700.4935 , 1571.2769 ,
       3544.644  , 3246.0518 , 1195.327  , 1015.4567 , 2243.4958 ,
       1707.0359 , 4748.2153 , 1564.1681 , 3515.3872 ,  712.10266,
       2265.2861 , 4392.0054 ,  899.1564 ,  856.66846, 2925.6582 ,
       3079.833  , 3039.157  ,  577.2381 , 4892.252  , 3035.3542 ,
        909.4257 , 3877.4175 , 1113.0638 , 1318.4141 , 1763.7898 ,
       2910.8052 , 2717.5818 ,  637.2786 , 3299.3545 ,  681.0755 ,
       1686.96   , 2178.9995 , 1172.3066 , 2045.2466 , 3089.477  ,
       1694.9451 , 1284.2762 ,  695.2841 , 1713.8043 ,  796.21295,
       1366.921  , 1086.2067 , 2637.3274 , 1694.74   , 1248.7252 ,
       3098.689  , 1025.9186 , 1147.8677 , 3517.7893 , 3143.1194 ,
        585.78546,  559.8675 , 1672.2162 , 1076.468  , 1701.5492 ,
       3911.1538 , 3022.122  , 1087.7236 ,  711.0371 , 1341.2351 ,
       1542.3086 ,  602.33856, 1311.3032 , 2385.468  , 3305.4875 ,
        586.52484,  922.06146,  559.7428 , 1221.0304 ,  904.25

#### Save Output Sample Data 

In [34]:
SampleData['Predictions']=y_pred
SampleData.to_csv('Sample_dashboard.csv',index=False)